# Project Proposal (Matt)

ADD TEXT HERE

# Project Code

## Data Extraction

### Flight Data (Myles & Sedra)

MK NOTES
1. Flight Data - Columns to Drop: 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'
2. Flight Data - Only Keep Rows with Following Airline Codes: 'UA', 'AA', 'US', 'WN', 'DL', 'VX'
3. Flight Data - Add Column for Total Delay (Add Together Departure & Arrival Delay Values)
4. Flight Data - Combine Date Value Columns into Single Date Column
5. Airline Data - Only Keep Rows with Following Airline Codes: 'UA', 'AA', 'US', 'WN', 'DL', 'VX'
6. Airport Data - Columns to Drop: 'LATITUDE', 'LONGITUDE'

In [1]:
#ADD CODE HERE

### Tweet Data (Natalie)

MK NOTES
1. Columns to Drop: 'negativereason', 'negativereason_confidence', 'airline_sentiment_gold', 'negativereason_gold', 'tweet_coord', 'tweet_location', 'user_timezone'
2. Convert Date Column to Only Include Date & Not Time
3. Convert Airline Name to Airline Code

In [2]:
#ADD CODE HERE

## Data Transformation

### Flight Data (Myles & Sedra)

MK NOTES
1. Airport Data - No Changes
2. Airline Data - No Changes
3. Flight Data - Use Group By & Filtering to Create Individual Data Frames for Delayed/Early/Cancelled/Diverted Flights for Each Airline (AIRLINE CODE MUST BE IN EACH DATAFRAME)

In [3]:
#ADD CODE HERE

In [4]:
import numpy as np
import pandas as pd
import datetime as dt
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [5]:
engine = create_engine("sqlite:///../February-2015-Airline-Delays-Tweets-ETL/database.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()

In [7]:
# reflect the tables
Base.prepare(engine, reflect=True)

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [9]:
inspector = inspect(engine)
inspector.get_table_names()
#columns = inspector.get_columns('Tweets')
#print(columns)

['Tweets']

In [10]:
data_tweets = pd.read_sql_table('Tweets',engine)
data_tweets = data_tweets.drop(['negativereason', 'negativereason_confidence', 'airline_sentiment_gold', 
                                'negativereason_gold', 
                                'tweet_coord', 'tweet_location', 'user_timezone'],axis=1)
data_tweets = data_tweets.dropna()

In [11]:
data_tweets.sort_values("tweet_created")
data_tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_created
0,567588278875213824,neutral,1.0000,Delta,JetBlueNews,0,@JetBlue's new CEO seeks the right balance to ...,2015-02-16 23:36:05 -0800
1,567590027375702016,negative,1.0000,Delta,nesi_1992,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,2015-02-16 23:43:02 -0800
2,567591480085463040,negative,1.0000,United,CPoutloud,0,@united yes. We waited in line for almost an h...,2015-02-16 23:48:48 -0800
3,567592368451248130,negative,1.0000,United,brenduch,0,@united the we got into the gate at IAH on tim...,2015-02-16 23:52:20 -0800
4,567594449874587648,negative,1.0000,Southwest,VahidESQ,0,@SouthwestAir its cool that my bags take a bit...,2015-02-17 00:00:36 -0800
5,567594579310825473,negative,1.0000,United,brenduch,0,@united and don't hope for me having a nicer f...,2015-02-17 00:01:07 -0800
6,567595670463205376,negative,1.0000,United,CRomerDome,0,@united I like delays less than you because I'...,2015-02-17 00:05:27 -0800
7,567614049425555457,negative,1.0000,United,JustOGG,0,"@united, link to current status of flights/air...",2015-02-17 01:18:29 -0800
8,567617081336950784,negative,1.0000,Southwest,mrshossruns,0,@SouthwestAir you guys there? Are we on hour 2...,2015-02-17 01:30:32 -0800
9,567617486703853568,negative,1.0000,United,feliciastoler,0,@united I tried 2 DM it would not go thru... n...,2015-02-17 01:32:09 -0800


In [12]:
data_tweets['airline'] = data_tweets["airline"].replace("United","UA")
data_tweets['airline'] = data_tweets["airline"].replace("Delta","DL")
data_tweets['airline'] = data_tweets["airline"].replace("Southwest","WN")
data_tweets['airline'] = data_tweets["airline"].replace("American","AA")
data_tweets['airline'] = data_tweets["airline"].replace("US Airways","US")
data_tweets['airline'] = data_tweets["airline"].replace("Virgin America","VX")
data_tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_created
0,567588278875213824,neutral,1.0000,DL,JetBlueNews,0,@JetBlue's new CEO seeks the right balance to ...,2015-02-16 23:36:05 -0800
1,567590027375702016,negative,1.0000,DL,nesi_1992,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,2015-02-16 23:43:02 -0800
2,567591480085463040,negative,1.0000,UA,CPoutloud,0,@united yes. We waited in line for almost an h...,2015-02-16 23:48:48 -0800
3,567592368451248130,negative,1.0000,UA,brenduch,0,@united the we got into the gate at IAH on tim...,2015-02-16 23:52:20 -0800
4,567594449874587648,negative,1.0000,WN,VahidESQ,0,@SouthwestAir its cool that my bags take a bit...,2015-02-17 00:00:36 -0800
5,567594579310825473,negative,1.0000,UA,brenduch,0,@united and don't hope for me having a nicer f...,2015-02-17 00:01:07 -0800
6,567595670463205376,negative,1.0000,UA,CRomerDome,0,@united I like delays less than you because I'...,2015-02-17 00:05:27 -0800
7,567614049425555457,negative,1.0000,UA,JustOGG,0,"@united, link to current status of flights/air...",2015-02-17 01:18:29 -0800
8,567617081336950784,negative,1.0000,WN,mrshossruns,0,@SouthwestAir you guys there? Are we on hour 2...,2015-02-17 01:30:32 -0800
9,567617486703853568,negative,1.0000,UA,feliciastoler,0,@united I tried 2 DM it would not go thru... n...,2015-02-17 01:32:09 -0800


In [13]:
data_tweets_new = data_tweets['tweet_created'].str.split(" ",n=2,expand=True)

In [14]:
data_tweets["tweet_date"] = data_tweets_new[0]
data_tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_created,tweet_date
0,567588278875213824,neutral,1.0000,DL,JetBlueNews,0,@JetBlue's new CEO seeks the right balance to ...,2015-02-16 23:36:05 -0800,2015-02-16
1,567590027375702016,negative,1.0000,DL,nesi_1992,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,2015-02-16 23:43:02 -0800,2015-02-16
2,567591480085463040,negative,1.0000,UA,CPoutloud,0,@united yes. We waited in line for almost an h...,2015-02-16 23:48:48 -0800,2015-02-16
3,567592368451248130,negative,1.0000,UA,brenduch,0,@united the we got into the gate at IAH on tim...,2015-02-16 23:52:20 -0800,2015-02-16
4,567594449874587648,negative,1.0000,WN,VahidESQ,0,@SouthwestAir its cool that my bags take a bit...,2015-02-17 00:00:36 -0800,2015-02-17
5,567594579310825473,negative,1.0000,UA,brenduch,0,@united and don't hope for me having a nicer f...,2015-02-17 00:01:07 -0800,2015-02-17
6,567595670463205376,negative,1.0000,UA,CRomerDome,0,@united I like delays less than you because I'...,2015-02-17 00:05:27 -0800,2015-02-17
7,567614049425555457,negative,1.0000,UA,JustOGG,0,"@united, link to current status of flights/air...",2015-02-17 01:18:29 -0800,2015-02-17
8,567617081336950784,negative,1.0000,WN,mrshossruns,0,@SouthwestAir you guys there? Are we on hour 2...,2015-02-17 01:30:32 -0800,2015-02-17
9,567617486703853568,negative,1.0000,UA,feliciastoler,0,@united I tried 2 DM it would not go thru... n...,2015-02-17 01:32:09 -0800,2015-02-17


In [15]:
data_tweets_time= data_tweets_new[1].str.split(":",n=2, expand=True)
data_tweets["tweet_time"]=data_tweets_time[0]  + ":" + data_tweets_time[1]
data_tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_created,tweet_date,tweet_time
0,567588278875213824,neutral,1.0000,DL,JetBlueNews,0,@JetBlue's new CEO seeks the right balance to ...,2015-02-16 23:36:05 -0800,2015-02-16,23:36
1,567590027375702016,negative,1.0000,DL,nesi_1992,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,2015-02-16 23:43:02 -0800,2015-02-16,23:43
2,567591480085463040,negative,1.0000,UA,CPoutloud,0,@united yes. We waited in line for almost an h...,2015-02-16 23:48:48 -0800,2015-02-16,23:48
3,567592368451248130,negative,1.0000,UA,brenduch,0,@united the we got into the gate at IAH on tim...,2015-02-16 23:52:20 -0800,2015-02-16,23:52
4,567594449874587648,negative,1.0000,WN,VahidESQ,0,@SouthwestAir its cool that my bags take a bit...,2015-02-17 00:00:36 -0800,2015-02-17,00:00
5,567594579310825473,negative,1.0000,UA,brenduch,0,@united and don't hope for me having a nicer f...,2015-02-17 00:01:07 -0800,2015-02-17,00:01
6,567595670463205376,negative,1.0000,UA,CRomerDome,0,@united I like delays less than you because I'...,2015-02-17 00:05:27 -0800,2015-02-17,00:05
7,567614049425555457,negative,1.0000,UA,JustOGG,0,"@united, link to current status of flights/air...",2015-02-17 01:18:29 -0800,2015-02-17,01:18
8,567617081336950784,negative,1.0000,WN,mrshossruns,0,@SouthwestAir you guys there? Are we on hour 2...,2015-02-17 01:30:32 -0800,2015-02-17,01:30
9,567617486703853568,negative,1.0000,UA,feliciastoler,0,@united I tried 2 DM it would not go thru... n...,2015-02-17 01:32:09 -0800,2015-02-17,01:32


In [16]:
data_tweets= data_tweets.drop(["tweet_created"],axis=1)
data_tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,airline,name,retweet_count,text,tweet_date,tweet_time
0,567588278875213824,neutral,1.0000,DL,JetBlueNews,0,@JetBlue's new CEO seeks the right balance to ...,2015-02-16,23:36
1,567590027375702016,negative,1.0000,DL,nesi_1992,0,@JetBlue is REALLY getting on my nerves !! 😡😡 ...,2015-02-16,23:43
2,567591480085463040,negative,1.0000,UA,CPoutloud,0,@united yes. We waited in line for almost an h...,2015-02-16,23:48
3,567592368451248130,negative,1.0000,UA,brenduch,0,@united the we got into the gate at IAH on tim...,2015-02-16,23:52
4,567594449874587648,negative,1.0000,WN,VahidESQ,0,@SouthwestAir its cool that my bags take a bit...,2015-02-17,00:00
5,567594579310825473,negative,1.0000,UA,brenduch,0,@united and don't hope for me having a nicer f...,2015-02-17,00:01
6,567595670463205376,negative,1.0000,UA,CRomerDome,0,@united I like delays less than you because I'...,2015-02-17,00:05
7,567614049425555457,negative,1.0000,UA,JustOGG,0,"@united, link to current status of flights/air...",2015-02-17,01:18
8,567617081336950784,negative,1.0000,WN,mrshossruns,0,@SouthwestAir you guys there? Are we on hour 2...,2015-02-17,01:30
9,567617486703853568,negative,1.0000,UA,feliciastoler,0,@united I tried 2 DM it would not go thru... n...,2015-02-17,01:32


In [17]:
#creating a groupby for airline codes
data_tweets_grouped = data_tweets.groupby(["airline"])

In [18]:
#creating separate dataframes for different airlines
AA_df = data_tweets_grouped.get_group("AA")

In [19]:
#Creating new dataframes based on type of review 

AA_grouped = AA_df.groupby("airline_sentiment")
AA_positive = AA_grouped.get_group("positive")
AA_positive=AA_positive.reset_index(drop=True)

In [20]:
#Creating new dataframes based on type of review 

AA_negative = AA_grouped.get_group("negative")
AA_negative=AA_negative.reset_index(drop=True)

In [21]:
#Creating new dataframes based on type of review 

AA_neutral = AA_grouped.get_group("neutral")
AA_neutral=AA_neutral.reset_index(drop=True)

In [22]:
#creating separate dataframes for different airlines
UA_df = data_tweets_grouped.get_group("UA")
#Creating new dataframes based on type of review 

UA_grouped = UA_df.groupby("airline_sentiment")
UA_positive = UA_grouped.get_group("positive")
UA_positive = UA_positive.reset_index(drop=True)

In [27]:
#Creating new dataframes based on type of review 

UA_negative = UA_grouped.get_group("negative")
UA_negative = UA_negative.reset_index(drop=True)

In [29]:
#Creating new dataframes based on type of review 

UA_neutral = UA_grouped.get_group("neutral")
UA_neutral = UA_neutral.reset_index(drop=True)

In [31]:
#creating separate dataframes for different airlines
US_df = data_tweets_grouped.get_group("US")
#Creating new dataframes based on type of review 

US_grouped = US_df.groupby("airline_sentiment")
US_positive = US_grouped.get_group("positive")
US_positive = US_positive.reset_index(drop=True)

In [34]:
#Creating new dataframes based on type of review 

US_negative = US_grouped.get_group("negative")
US_negative = US_negative.reset_index(drop=True)

In [36]:
#Creating new dataframes based on type of review 

US_neutral = US_grouped.get_group("neutral")
US_neutral = US_neutral.reset_index(drop=True)

In [39]:
#creating separate dataframes for different airlines
WN_df = data_tweets_grouped.get_group("WN")
#Creating new dataframes based on type of review 

WN_grouped = WN_df.groupby("airline_sentiment")
WN_positive = WN_grouped.get_group("positive")
WN_positive = WN_positive.reset_index(drop=True)

In [41]:
#Creating new dataframes based on type of review 

WN_negative = WN_grouped.get_group("negative")
WN_negative = WN_negative.reset_index(drop=True)

In [43]:
#Creating new dataframes based on type of review 

WN_neutral = WN_grouped.get_group("neutral")
WN_neutral = WN_neutral.reset_index(drop=True)

In [45]:
#creating separate dataframes for different airlines
DL_df = data_tweets_grouped.get_group("DL")
#Creating new dataframes based on type of review 

DL_grouped = DL_df.groupby("airline_sentiment")
DL_positive = DL_grouped.get_group("positive")
DL_positive = DL_positive.reset_index(drop=True)

In [47]:
#Creating new dataframes based on type of review 

DL_negative = DL_grouped.get_group("negative")
DL_negative = DL_negative.reset_index(drop=True)

In [49]:
#Creating new dataframes based on type of review 

DL_neutral = DL_grouped.get_group("neutral")
DL_neutral = DL_neutral.reset_index(drop=True)

In [51]:
#creating separate dataframes for different airlines
VX_df = data_tweets_grouped.get_group("VX")
#Creating new dataframes based on type of review 

VX_grouped = VX_df.groupby("airline_sentiment")
VX_positive = VX_grouped.get_group("positive")
VX_positive = VX_positive.reset_index(drop=True)

In [55]:
#Creating new dataframes based on type of review 

VX_negative = VX_grouped.get_group("negative")
VX_negative = VX_negative.reset_index(drop=True)

In [57]:
#Creating new dataframes based on type of review 

VX_neutral = VX_grouped.get_group("neutral")
VX_neutral = VX_neutral.reset_index(drop=True)

### Tweet Data (Natalie)

MK NOTES
1. Use Group By & Filtering to Create Data Frames of Positive/Negative/Neutral Tweets for Each Airline (AIRLINE CODE MUST BE IN EACH DATAFRAME)

In [23]:
#ADD CODE HERE

## Data Loading (Matt)

MK Notes
1. Create ERD for Completed Database
2. Create Database in PGAdmin
3. Use SQL Alchemy to Load Dataframes into Tables & Relate by Keys

In [24]:
#ADD CODE HERE

# Project Analysis

## Data Extraction

### Flight Data (Myles & Sedra)

ADD TEXT HERE

### Tweet Data (Natalie)

ADD TEXT HERE

## Data Transformation

### Flight Data (Myles & Sedra)

ADD TEXT HERE

### Tweet Data (Natalie)

ADD TEXT HERE

## Data Loading (Matt)

ADD TEXT HERE